In [5]:
import pandas as pd
import numpy as np
import openpyxl
import datetime

营养素的库存管理复杂，经手人多，账套混乱，操作原始。核对起来相当痛苦。

为了做一个全局管理脚本，在这理一下思路。

首先，营养素有住院部和服务部2个部门在同时售卖。
住院部的库存在食堂仓库，需要人工清点。服务部的库存在服务部，需要人工清点。

首先，营养素的管理有两个系统。
## 总系统：百威进销存系统
百威进销存系统记录库存，这个系统减去当天服务部售卖的营养素，但是住院部售卖的营养素需要到月底根据手工统计减去。
### 医院的记账系统 
医院的记账系统只记录住院部的售卖情况。不记录库存。
住院部的工作除我以外还有两人参与：

住院部订单员：下达订单，通知仓库管理员来取货。自己维护一张订单表格

仓库管理员：接到通知时在当日内取货。并计入医院的记账系统
#### 出错可能
食堂管理员少拿错拿营养素，记错帐
#### 开发功能需求
定期核对前一日食堂管理员的医院帐和订单员维护的帐
### 服务部的记账方法 
服务部的帐月末结入百威系统，当月售卖
#### 出错可能
服务部在系统内记账，记漏记错
#### 开发功能需求
记录服务部每日库存差和挂账是否相等

#### 综合需求
昨日百威进销存系统的结存数量 = 仓库库存+服务部库存+服务部挂账+核对后的住院部订单
## 库存管理：

### 入库：
入库由我报数量下订单给采购员，采购员送货后。服务部将其录入。由于服务部有退货可能，所以需在服务部验收后将数量报给我。
### 借调
服务部库存不够时，由我向食堂调货。这时，需在系统内调整库存。
## 保质期管理：
没想好怎么管，各种批号的营养素掺在一起 麻烦炸了先记录下来 功能保留 （用概率论猜？）
## 回溯查询：
要有每次查询要有历史记录，可以随时回档。使用git进行版本管理
## 使用场景
每天早上
我：核对订单员和医院记账系统是否对应

系统：读取订单员表格 数据清洗整理 统计出每天每个品卖出几个 去记账系统看

我：准备一张表，填入核对日期，入库量，失效日期，百威结存数量，仓库库存，服务部库存（允许填不变），昨日借调数量，服务部挂账，住院部售卖数量

系统：核对数据，提示出错商品错了几个。并更新明天的表格

我：git add git commit git push



### 清洗数据

In [6]:
zhuyuandata = pd.read_excel('yys.xls',header=1,sheet_name=1)
zhuyuandata =zhuyuandata.iloc[:len(zhuyuandata)-1,]
zhuyuandata = zhuyuandata.dropna(axis=0,how='all')
zhuyuandata.head()

,日期,病区,床号,姓名,住院号,出生年月,营养制剂名称,数量,费用,配送人员
0,9.01,B3X,3.0,夏香凤,380958.0,1943-07-08 00:00:00,乳清蛋白,1.0,188,黄海琴
1,,B3X,19.0,彭碧华,408027.0,1953-12-16 00:00:00,乳清蛋白,1.0,188,黄海琴
2,9.02,B3X,21.0,袁其红,242889.0,1953-03-03 00:00:00,乳清蛋白,1.0,188,王艳/朱
3,9.03,A6D,33.0,郑荷香,357954.0,1952/6/27,乳清蛋白,1.0,188,王艳/朱
4,NaN,B3X,18.0,张邦才,404135.0,1956-09-21 00:00:00,乳清蛋白,1.0,188,王艳/朱


#### 表头清洗

In [7]:
zhuyuandata.columns=zhuyuandata.columns.map(lambda x:str(x).strip(' '))
zhuyuandata.columns

Index(['日期', '病区', '床号', '姓名', '住院号', '出生年月', '营养制剂名称', '数量', '费用', '配送人员'], dtype='object')

将完全为空的格子转换为NaN

In [8]:
a = (zhuyuandata.isnull()|zhuyuandata.applymap(lambda x:str(x).isspace()))
#zhuyuandata_null = zhuyuandata[a]
#huyuandata_null
zhuyuandata_not_null =zhuyuandata[~a]
#删除两边的空格
zhuyuandata_not_null=zhuyuandata_not_null.applymap(lambda x:str(x).strip(' '))
zhuyuandata_not_null.head()


,日期,病区,床号,姓名,住院号,出生年月,营养制剂名称,数量,费用,配送人员
0,9.01,B3X,3.0,夏香凤,380958.0,1943-07-08 00:00:00,乳清蛋白,1.0,188,黄海琴
1,nan,B3X,19.0,彭碧华,408027.0,1953-12-16 00:00:00,乳清蛋白,1.0,188,黄海琴
2,9.02,B3X,21.0,袁其红,242889.0,1953-03-03 00:00:00,乳清蛋白,1.0,188,王艳/朱
3,9.03,A6D,33.0,郑荷香,357954.0,1952/6/27,乳清蛋白,1.0,188,王艳/朱
4,nan,B3X,18.0,张邦才,404135.0,1956-09-21 00:00:00,乳清蛋白,1.0,188,王艳/朱


In [9]:
for i in range(75):
    if zhuyuandata_not_null.loc[i,'日期'] == "nan":
        zhuyuandata_not_null.loc[i,'日期']=zhuyuandata_not_null.loc[i-1,'日期']
    if zhuyuandata_not_null.loc[i,'姓名'] == "nan":
        zhuyuandata_not_null.loc[i,'姓名']=zhuyuandata_not_null.loc[i-1,'姓名']
    if zhuyuandata_not_null.loc[i,'病区'] == "nan":
        zhuyuandata_not_null.loc[i,'病区']=zhuyuandata_not_null.loc[i-1,'病区']
    if zhuyuandata_not_null.loc[i,'床号'] == "nan":
        zhuyuandata_not_null.loc[i,'床号']=zhuyuandata_not_null.loc[i-1,'床号']
    if zhuyuandata_not_null.loc[i,'住院号'] == "nan":
        zhuyuandata_not_null.loc[i,'住院号']=zhuyuandata_not_null.loc[i-1,'住院号']
    if zhuyuandata_not_null.loc[i,'出生年月'] == "nan":
        zhuyuandata_not_null.loc[i,'出生年月']=zhuyuandata_not_null.loc[i-1,'出生年月']
zhuyuandata_not_null.head()


,日期,病区,床号,姓名,住院号,出生年月,营养制剂名称,数量,费用,配送人员
0,9.01,B3X,3.0,夏香凤,380958.0,1943-07-08 00:00:00,乳清蛋白,1.0,188,黄海琴
1,9.01,B3X,19.0,彭碧华,408027.0,1953-12-16 00:00:00,乳清蛋白,1.0,188,黄海琴
2,9.02,B3X,21.0,袁其红,242889.0,1953-03-03 00:00:00,乳清蛋白,1.0,188,王艳/朱
3,9.03,A6D,33.0,郑荷香,357954.0,1952/6/27,乳清蛋白,1.0,188,王艳/朱
4,9.03,B3X,18.0,张邦才,404135.0,1956-09-21 00:00:00,乳清蛋白,1.0,188,王艳/朱


进行强制的数据转换

In [10]:
zhuyuandata_not_null["数量"] = zhuyuandata_not_null["数量"].astype("float")
zhuyuandata_not_null["日期"] = zhuyuandata_not_null["日期"].astype("str")
zhuyuandata_not_null.dtypes

日期         object
病区         object
床号         object
姓名         object
住院号        object
出生年月       object
营养制剂名称     object
数量        float64
费用         object
配送人员       object
dtype: object

将费用字段中带退货的数据单独提取出来
先复制一个df 用来存退货的数据

In [11]:

df_returns =  pd.DataFrame( columns=zhuyuandata_not_null.columns)
df_returns
df_sells = pd.DataFrame( columns=zhuyuandata_not_null.columns)


In [12]:
for i in range(len(zhuyuandata_not_null)):
    if zhuyuandata_not_null.loc[i,"费用"]=='退':
        df_returns = df_returns.append(zhuyuandata_not_null.loc[i],ignore_index=True)
        #zhuyuandata_not_null = zhuyuandata_not_null.drop[i]
    else:
        df_sells = df_sells.append(zhuyuandata_not_null.loc[i],ignore_index=True) 
df_returns

,日期,病区,床号,姓名,住院号,出生年月,营养制剂名称,数量,费用,配送人员
0,9.9,A6D,42.0,陈荷女,409076.0,1935-08-20 00:00:00,伊能素,1.0,退,王艳/朱
1,9.9,B3X,30.0,刘君央,409196.0,1964-12-10 00:00:00,伊支素,1.0,退,王艳/朱
2,9.24,B7X,14.0,翟三妹,410881.0,1938-11-25 00:00:00,乳清蛋白,1.0,退,王艳/付
3,9.24,B7X,14.0,翟三妹,410881.0,1938-11-25 00:00:00,伊能素,1.0,退,王艳/付


将费用转换成int类型，再显示df_sell数组

In [13]:
df_sells["费用"] = df_sells["费用"].astype("int")
df_sells.head()

,日期,病区,床号,姓名,住院号,出生年月,营养制剂名称,数量,费用,配送人员
0,9.01,B3X,3.0,夏香凤,380958.0,1943-07-08 00:00:00,乳清蛋白,1.0,188,黄海琴
1,9.01,B3X,19.0,彭碧华,408027.0,1953-12-16 00:00:00,乳清蛋白,1.0,188,黄海琴
2,9.02,B3X,21.0,袁其红,242889.0,1953-03-03 00:00:00,乳清蛋白,1.0,188,王艳/朱
3,9.03,A6D,33.0,郑荷香,357954.0,1952/6/27,乳清蛋白,1.0,188,王艳/朱
4,9.03,B3X,18.0,张邦才,404135.0,1956-09-21 00:00:00,乳清蛋白,1.0,188,王艳/朱


### 日期清洗
先把日期转成字符串 再以小数点为分界：前一部分为月，后一部分为日。
根据日期筛选

In [20]:
for i in df_sells['日期']:
    print(i.split('.'))

['9', '01']
['9', '01']
['9', '02']
['9', '03']
['9', '03']
['9', '03']
['9', '03']
['9', '03']
['9', '05']
['9', '05']
['9', '05']
['9', '05']
['9', '06']
['9', '06']
['9', '06']
['9', '06']
['9', '8']
['9', '8']
['9', '8']
['9', '9']
['9', '9']
['9', '9']
['9', '9']
['9', '10']
['9', '10']
['9', '11']
['9', '11']
['9', '11']
['9', '11']
['9', '11']
['9', '12']
['9', '13']
['9', '13']
['9', '17']
['9', '17']
['9', '17']
['9', '17']
['9', '17']
['9', '17']
['9', '18']
['9', '18']
['9', '19']
['9', '19']
['9', '19']
['9', '19']
['9', '19']
['9', '20']
['9', '20']
['9', '20']
['9', '20']
['9', '20']
['9', '20']
['9', '25']
['9', '26']
['9', '26']
['9', '26']
['9', '26']
['9', '26']
['9', '26']
['9', '27']
['9', '27']
['9', '27']
['9', '27']
['9', '27']
['9', '29']
['9', '29']
['9', '30']
['9', '30']
['9', '30']
['9', '30']
['9', '30']


In [15]:
df_sells[df_sells['日期']>'9.2']

,日期,病区,床号,姓名,住院号,出生年月,营养制剂名称,数量,费用,配送人员
16,9.8,A3D,16.0,伍邦香,398208.0,1954-04-07 00:00:00,乳清蛋白,1.0,188,黄海琴
17,9.8,A3D,16.0,伍邦香,398208.0,1954-04-07 00:00:00,伊肽素,1.0,198,黄海琴
18,9.8,A3D,16.0,伍邦香,398208.0,1954-04-07 00:00:00,匀浆膳,1.0,68,黄海琴
19,9.9,B3X,18.0,张邦才,404135.0,1956-09-21 00:00:00,乳清蛋白,1.0,188,王艳/朱
20,9.9,B3X,18.0,张邦才,404135.0,1956-09-21 00:00:00,伊能素,1.0,380,王艳/朱
21,9.9,B3X,3.0,夏香凤,380958.0,1943-07-08 00:00:00,乳清蛋白,1.0,188,王艳/朱
22,9.9,B3X,16.0,张金凤,406706.0,1943-08-08 00:00:00,乳清蛋白,1.0,188,王艳/朱
46,9.20,A6D,13.0,王小兰,410401.0,1942-01-19 00:00:00,乳清蛋白,1.0,188,王艳/朱
47,9.20,A6D,13.0,王小兰,410401.0,1942-01-19 00:00:00,伊全素,1.0,98,王艳/朱
48,9.20,A6D,14.0,王云喜,366439.0,1948-11-17 00:00:00,乳清蛋白,1.0,188,王艳/朱


groupby归类卖出的数量

In [16]:
grouped = df_sells.groupby('营养制剂名称')

grouped_date = grouped['数量','费用']
grouped_date.sum()

,数量,费用
营养制剂名称,,
乳清蛋白,37.0,6956
伊低素,2.0,304
伊全素,13.0,1274
伊支素,1.0,212
伊肽素,5.0,990
伊能素,10.0,3800
匀浆膳,3.0,204


# 检查模块

# 数据流通
将手工统计的数据取最新填入excel